# Implementation of the Newmark-β time-stepping

In this notebook, we present an alternative imlementation of the Newmark-β time-stepping technique that was previously introduced through [Jérémy Bleyer's code](https://comet-fenics.readthedocs.io/en/latest/demo/elastodynamics/demo_elastodynamics.py.html). We adopt here an approach that is more “matrix-oriented”: `FEniCS` is used to assemble the matrices of the problem, discretized in space. Once these matrices are assembled, we write the various linear algebra operations explicitly.

We will explore various versions of the Newmark method, including the so-called “explicit” version.

We consider a cantilever beam of size `Lx × Ly` (2D, plane stress) or `Lx × Ly × Lz` (3D), fixed at its `x = 0` end, and subjected to a uniform traction at `x = Lx`. The direction of the traction can be adapted (see parameter `T_dir` below).

**Note:** when you first run this notebook, you will get *many* error messages caused by `raise RuntimeException()` statements. You need to replace each of these statements with the correct code for the simulation to run.

In [ ]:
import dolfin
import numpy as np
import matplotlib.pyplot as plt
import time
import warnings

# Show all warnings, multiple times if necessary
warnings.filterwarnings("default")

dolfin.parameters["form_compiler"]["cpp_optimize"] = True
dolfin.parameters["form_compiler"]["optimize"] = True

## Parameters of the simulation

Except otherwise explicitly stated, code outside the present section need not be modified.

### Geometry

In [ ]:
dim = 2 # Number of spatial dimensions

Lx = 1.0
Ly = 0.1
Lz = 0.04 # This is used only if dim == 3

Ny = 10 # Number of elements in the y-direction

### Material parameters

In [ ]:
E  = 1000.
nu = 0.3
rho = dolfin.Constant(1.)
eta_m = dolfin.Constant(0.)
eta_k = dolfin.Constant(0.)

### Loading parameters

A time-dependent traction is applied at the $x = L_x$ face as follows
\begin{equation}
\vec T = \begin{cases}
\displaystyle\frac{t}{t_{\mathrm{c}}}\vec T_{\mathrm{max}} & t \leq t_{\mathrm{c}}\\[.2em]
\vec 0 & t > t_{\mathrm{c}}
\end{cases}
\end{equation}

where $t_{\mathrm{c}}$ is a “cut-off” time. Note that depending on the direction of the applied traction, we will need to select different values for `t_c` and `T_max`.

In [ ]:
t_c = 0.1
T_dir = 1  # Direction of the applied traction: x → 0, y → 1, z → 2
T_max = 1.

### Discretization parameters

In [ ]:
t_end = 4. # End of simulation
num_steps  = 100 # Total number of steps

Parameters of the Newmark-β method.

In [ ]:
beta = 0.
gamma = 0.5

gamma = 0.5
beta = 0.5*gamma

Set this flag to `true` if you want to use a “lumped” mass matrix, which is diagonal. Should only be used in the case of the “explicit” (centered-differences) version of the scheme.

In [ ]:
use_lumped_mass = False

## Validation of inputs

### Validation of the `use_lumped_mass` flag

In [ ]:
beta_exp = np.inf; raise RuntimeError("insert expected value of β")
gamma_exp = np.inf; raise RuntimeError("insert expected value of γ")

atol = 1e-12

test_beta = abs(float(beta)-beta_exp) < atol
test_gamma = abs(float(gamma)-gamma_exp) < atol
if use_lumped_mass and not (test_beta and test_gamma):
    raise RuntimeError("cannot use lumped mass in this case")

**Question:** insert in the cell above the expected values of β and γ.

**Question:** why did we specify an *absolute* tolerance only?

## Mesh generation

In [ ]:
p1 = dolfin.Point(0., 0., 0.)
p2 = dolfin.Point(1.0, 0.1, 0.04)

Nx = int(Ny*Lx/Ly)

if dim == 2:
    mesh = dolfin.RectangleMesh(p1, p2, Nx, Ny)
elif dim == 3:
    Nz = int(Ny*Lz/Ly)
    mesh = dolfin.BoxMesh(p1, p2, Nx, Ny, Nz)
else:
    raise ValueError("dim must be 2 or 3 (was {})".format(dim))

In [ ]:
dolfin.plot(mesh)

In [ ]:
left = dolfin.CompiledSubDomain("on_boundary && near(x[0],0)")
right = dolfin.CompiledSubDomain("on_boundary && near(x[0], L)", L=p2.x())
top = dolfin.CompiledSubDomain("on_boundary && near(x[1], H)", H=p2.y())
bottom = dolfin.CompiledSubDomain("on_boundary && near(x[1], 0)")

boundary_indices = {"left": 0, "right": 1, "top": 2, "bottom": 3}
boundary_markers = dolfin.MeshFunction("size_t", mesh, dim=1, value=0)
left.mark(boundary_markers, boundary_indices["left"])
right.mark(boundary_markers, boundary_indices["right"])
right.mark(boundary_markers, boundary_indices["right"])
top.mark(boundary_markers, boundary_indices["top"])
bottom.mark(boundary_markers, boundary_indices["bottom"])

ds = dolfin.ds(domain=mesh, subdomain_data=boundary_markers)
dx = dolfin.dx(domain=mesh)

### Validation of time-step

**Question**: in the cell below, compute the critical time step $\Delta t_{\mathrm{crit}}$ (see p. 41 in [the slides](https://github.com/msolides2020/MU5MES01-2020/blob/master/03-Dynamics/cours_dynamique.pdf)).

**Hint**: use the command [`mesh.rmin()`](https://fenicsproject.org/olddocs/dolfin/latest/cpp/da/dfc/classdolfin_1_1Mesh.html#a6ddfafebe68a370a5555a370fdbcdbbd).

In [ ]:
dt = t_end/num_steps

raise RuntimeError("implement formula for Δt_crit")
dt_crit = np.inf

In [ ]:
if dt > dt_crit:
    warnings.warn("simulation might be unstable")

**Question:** is this warning always necessary? Modify the above test accordingly.

## Space discretization

In [ ]:
V = dolfin.VectorFunctionSpace(mesh, "CG", 1)
Vsig = dolfin.TensorFunctionSpace(mesh, "DG", 0)

In [ ]:
zero = dolfin.Constant(dim*(0.,))
bcs = [dolfin.DirichletBC(V, zero, left)]

if T_dir == 0:
    bcs += [dolfin.DirichletBC(V.sub(1), 0, top),
            dolfin.DirichletBC(V.sub(1), 0, bottom)]

**Question**: comment on the additional boundary conditions when `T_dir == 0`. Why are these boundary conditions not applied for `T_dir == 1`?

In [ ]:
mu = dolfin.Constant(E/(2.*(1.+nu)))
lambda_ = dolfin.Constant(E*nu/((1.+nu)*(1.-2.*nu)))
if dim == 2:
    lambda_ = 2*lambda_*mu/(lambda_+2*mu)

In [ ]:
expr = dim*["0",]
expr[T_dir] = "t <= t_c ? T_max*t/t_c : 0"

traction = dolfin.Expression(expr, t=0, t_c=t_c, T_max=T_max, degree=0)

**Question:** why did we specify `degree=0` in the above `dolfin.Expression`?

In [ ]:
def local_project(v, V, u=None):
    """Element-wise projection using LocalSolver"""
    dv = dolfin.TrialFunction(V)
    v_ = dolfin.TestFunction(V)
    a_proj = dolfin.inner(dv, v_)*dx
    b_proj = dolfin.inner(v, v_)*dx
    solver = dolfin.LocalSolver(a_proj, b_proj)
    solver.factorize()
    if u is None:
        u = dolfin.Function(V)
        solver.solve_local_rhs(u)
        return u
    else:
        solver.solve_local_rhs(u)
        return

In [ ]:
I2 = dolfin.Identity(dim)

def stress_strain(eps):
    return lambda_*dolfin.tr(eps)*I2+2.*mu*eps

def strain_displacement(u):
    return dolfin.sym(dolfin.grad(u))

def mass(u, v):
    return rho*dolfin.inner(u, v)*dx

def stiffness(u, v):
    return dolfin.inner(stress_strain(strain_displacement(u)),
                        strain_displacement(v))*dx

def damping(u, v):
    return eta_m*mass(u, v)+eta_k*stiffness(u, v)

def p_ext(u):
    return dolfin.dot(u, traction)*ds(boundary_indices["right"])

In [ ]:
u = dolfin.TrialFunction(V)
v = dolfin.TestFunction(V)

C = dolfin.assemble(damping(u, v))
K = dolfin.assemble(stiffness(u, v))

for bc in bcs:
    bc.apply(C)
    bc.apply(K)

The mass matrix is modified as follows
\begin{equation*}
M\leftarrow M+\gamma\Delta t\,C+\beta\Delta t^2\,K.
\end{equation*}

**Question:** why did we do that?

The cell below computes the modified mass matrix and defines the function `compute_acceleration(f, a)` that performs the operation: `a ← M⁻¹⋅f`, where `a` and `f` are two vectors.

**Note:** for the computation of the lumped mass matrix, we use the function [`dolfin.action`](https://fenics.readthedocs.io/projects/ufl/en/latest/manual/form_language.html#action-of-a-form-on-a-function).

In [ ]:
one_half = dolfin.Constant(0.5)

if use_lumped_mass:
    # Compute the sum of the rows of the mass matrix
    m = mass(u, v)+one_half*dt*damping(u, v)
    ones = dolfin.Constant(dim*(1.,))
    M_lumped = dolfin.assemble(dolfin.action(m, ones))
    M_lumped_inv = 1./M_lumped.get_local()
    
    def compute_acceleration(f_vec, a_vec):
        a_vec[:] = M_lumped_inv*f[:]
        for bc in bcs:
            bc.apply(a_vec)
        
else:
    M = dolfin.assemble(mass(u, v))
    M += gamma*dt*C
    M += beta*dt**2*K
    for bc in bcs:
        bc.apply(M)
    solver = dolfin.LUSolver(M)
    
    def compute_acceleration(f_vec, a_vec):
        solver.solve(a_vec, f_vec)

## Time discretization

In [ ]:
num_functions = 5
functions = [dolfin.Function(V) for i in range(num_functions)]
u, v, a, Ku, Cv = functions
u_vec, v_vec, a_vec, Ku_vec, Cv_vec = [func.vector() for func in functions]

In [ ]:
coords_tip = p2.array()[:dim]
coords_tip[1:] *= 0.5

times = dt*np.arange(num_steps+1, dtype=np.float64)
displ_tip = np.zeros_like(times)

energies = np.zeros((num_steps+1, 4), dtype=np.float64)
E_damp = 0
E_ext = 0

sig = dolfin.Function(Vsig, name="sigma")
xdmf_file = dolfin.XDMFFile("elastodynamics-results.xdmf")
xdmf_file.parameters["flush_output"] = True
xdmf_file.parameters["functions_share_mesh"] = True
xdmf_file.parameters["rewrite_function_mesh"] = False

**Question:** in the cell below:

- update `u_vec` and `v_vec` (predictor step)
- compute `f_vec`, which is the vector of (external + internal) nodal forces (the present code computes the external part only)
- update `u_vec` and `v_vec` (corrector step)

**Hint 1:** use the function `y.axpy(α, x)` to perform the operation `y ← α⋅x + y`, where `x` and `y` are two vectors and `α` is a scalar.

**Hint 2:** use the function `M.mult(x, y)` to perform the operation `y ← M⋅x`, where `x` and `y` are two vectors and `M` is a matrix.

In [ ]:
u_vec.zero()
v_vec.zero()
a_vec.zero()
Ku_vec.zero()

time_solve = 0.

for n in range(num_steps):
    # Predictor step
    raise RuntimeError("update u_vec and v_vec (predictor step)")

    # Update acceleration
    traction.t = (n+1)*dt
    f = dolfin.assemble(p_ext(dolfin.TestFunction(V)))
    
    raise RuntimeError("add contribution of internal forces to f")
    
    for bc in bcs:
        bc.apply(f)
    
    t1 = time.perf_counter()
    compute_acceleration(f, a_vec)
    t2 = time.perf_counter()
    time_solve += t2-t1
    
    # Corrector step
    raise RuntimeError("update u_vec and v_vec (corrector step)")
    
    displ_tip[n+1] = u(*coords_tip)[T_dir]
    
    E_elas = dolfin.assemble(0.5*stiffness(u, u))
    E_kin = dolfin.assemble(0.5*mass(v, v))
    E_damp += dt*dolfin.assemble(damping(v, v))
    E_tot = E_elas+E_kin+E_damp
    energies[n+1, :] = np.array([E_elas, E_kin, E_damp, E_tot])
    
    local_project(stress_strain(strain_displacement(u)), Vsig, sig)

    # Save solution to XDMF format
    xdmf_file.write(u, n)
    xdmf_file.write(sig, n)

print(time_solve)

In [ ]:
dolfin.plot(u, mode="displacement")

In [ ]:
plt.figure()
plt.plot(times, displ_tip, '-')
plt.xlabel("Time")
plt.ylabel("Tip displacement")
plt.savefig("tip_displacement.png")

In [ ]:
plt.figure()
plt.plot(times, energies)
plt.legend(("elastic", "kinetic", "damping", "total"))
plt.xlabel("Time")
plt.ylabel("Energies")
plt.show()